In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ta import add_all_ta_features
from ta.utils import dropna
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
import yfinance as yf

In [2]:
sp500_data = pd.read_csv("dataset/sp500_companies.csv")
sp500_data.head()

,Exchange,Symbol,Shortname,Longname,Sector,Industry,Currentprice,Marketcap,Ebitda,Revenuegrowth,City,State,Country,Fulltimeemployees,Longbusinesssummary,Weight
0,NMS,AAPL,Apple Inc.,Apple Inc.,Technology,Consumer Electronics,151.60,2487210213376,1.306340e+11,0.086,Cupertino,CA,United States,154000.0,"Apple Inc. designs, manufactures, and markets ...",0.078540
1,NMS,MSFT,Microsoft Corporation,Microsoft Corporation,Technology,Software—Infrastructure,251.90,1891260039168,9.498300e+10,0.184,Redmond,WA,United States,181000.0,"Microsoft Corporation develops, licenses, and ...",0.059721
2,NMS,TSLA,"Tesla, Inc.","Tesla, Inc.",Consumer Cyclical,Auto Manufacturers,776.58,779888230400,1.270200e+10,0.805,Austin,TX,United States,99290.0,"Tesla, Inc. designs, develops, manufactures, l...",0.024627
3,NYQ,BRK-B,Berkshire Hathaway Inc. New,Berkshire Hathaway Inc.,Financial Services,Insurance—Diversified,285.31,638886084608,1.255810e+11,0.115,Omaha,NE,United States,372000.0,"Berkshire Hathaway Inc., through its subsidiar...",0.020174
4,NYQ,UNH,UnitedHealth Group Incorporated,UnitedHealth Group Incorporated,Healthcare,Healthcare Plans,531.59,500678590464,2.639100e+10,0.142,Minnetonka,MN,United States,350000.0,UnitedHealth Group Incorporated operates as a ...,0.015810


In [3]:
# Handle Missing States
sp500_data[sp500_data["State"].isnull()].head()

,Exchange,Symbol,Shortname,Longname,Sector,Industry,Currentprice,Marketcap,Ebitda,Revenuegrowth,City,State,Country,Fulltimeemployees,Longbusinesssummary,Weight
32,NYQ,ACN,Accenture plc,Accenture plc,Technology,Information Technology Services,285.18,180534902784,1.004265e+10,0.218,Dublin,NaN,Ireland,710000.0,"Accenture plc, a professional services company...",0.005701
45,NYQ,LIN,Linde plc,Linde plc,Basic Materials,Specialty Chemicals,286.62,146908512256,1.036800e+10,0.134,Woking,NaN,United Kingdom,72327.0,Linde plc operates as an industrial gas and en...,0.004639
54,NYQ,MDT,Medtronic plc.,Medtronic plc,Healthcare,Medical Devices,91.90,123565064192,9.609000e+09,-0.002,Dublin,NaN,Ireland,90000.0,"Medtronic plc develops, manufactures, distribu...",0.003902
83,NYQ,CB,Chubb Limited,Chubb Limited,Financial Services,Insurance—Property & Casualty,188.28,81099915264,7.998000e+09,-0.034,Zurich,NaN,Switzerland,31000.0,Chubb Limited provides insurance and reinsuran...,0.002561
120,NYQ,AON,Aon plc,Aon plc,Financial Services,Insurance Brokers,279.24,61525504000,3.936000e+09,0.041,Dublin,NaN,Ireland,50000.0,"Aon plc, a professional services firm, provide...",0.001943


In [4]:
# Drop Missing States (Basically Outside of the US)
sp500_data = sp500_data = sp500_data.dropna(subset="State")

In [5]:
# Handle Missing Revenue Growth using yahoo finance
sp500_data[sp500_data["Revenuegrowth"].isnull()].head()

,Exchange,Symbol,Shortname,Longname,Sector,Industry,Currentprice,Marketcap,Ebitda,Revenuegrowth,City,State,Country,Fulltimeemployees,Longbusinesssummary,Weight
52,NYQ,CVS,CVS Health Corporation,CVS Health Corporation,Healthcare,Healthcare Plans,95.31,125814915072,1.828900e+10,NaN,Woonsocket,RI,United States,216000.0,CVS Health Corporation provides health service...,0.003973


In [9]:
# Get CVS data
cvs_ticker = yf.Ticker("CVS")

financials = cvs_ticker.quarterly_financials

if 'Total Revenue' in financials.index:
  revenue = financials.loc["Total Revenue"]
  revenue_growth = revenue.pct_change()

recent_growth = revenue_growth.dropna()
if not recent_growth.empty:
    latest_growth = recent_growth.iloc[0] 

if not recent_growth.empty:
    sp500_data.loc[sp500_data['Symbol'] == 'CVS', 'Revenuegrowth'] = latest_growth
else:
    print("No valid recent growth data available.")

print(revenue_growth)

2024-03-31         NaN
2023-12-31    0.060789
2023-09-30   -0.043160
2023-06-30   -0.009391
2023-03-31   -0.040969
2022-12-31    0.000000
2022-09-30    0.000000
Name: Total Revenue, dtype: float64
2024-03-31         NaN
2023-12-31    0.060789
2023-09-30   -0.043160
2023-06-30   -0.009391
2023-03-31   -0.040969
2022-12-31    0.000000
2022-09-30    0.000000
Name: Total Revenue, dtype: float64


C:\Users\wrjod\AppData\Local\Temp\ipykernel_29392\3273248409.py:8: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  revenue_growth = revenue.pct_change()
C:\Users\wrjod\AppData\Local\Temp\ipykernel_29392\3273248409.py:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  revenue_growth = revenue.pct_change()


In [10]:
sp500_data[sp500_data["Symbol"] == "CVS"]

,Exchange,Symbol,Shortname,Longname,Sector,Industry,Currentprice,Marketcap,Ebitda,Revenuegrowth,City,State,Country,Fulltimeemployees,Longbusinesssummary,Weight
52,NYQ,CVS,CVS Health Corporation,CVS Health Corporation,Healthcare,Healthcare Plans,95.31,125814915072,1.828900e+10,0.060789,Woonsocket,RI,United States,216000.0,CVS Health Corporation provides health service...,0.003973
